# Lightweight Fine-Tuning Project

TODO: In this cell, describe your choices for each of the following

* PEFT technique: I am using LoRA due to the fact that it is the most universally compatible PEFT technique. I am also the most familiar with it, which will make my coding and debugging process much more efficient
* Model: I am using distilbert for my model. We have used distilbert previously throughout the course and I find it to be very effective when it comes to classifying text.
* Evaluation approach: Evaluate method with a Hugging Face trainer
* Fine-tuning dataset: Hate-Offensive Speech Dataset

## Loading and Evaluating a Foundation Model

TODO: In the cells below, load your chosen pre-trained Hugging Face model and evaluate its performance prior to fine-tuning. This step includes loading an appropriate tokenizer and dataset.

In [20]:
%pip install pandas
%pip install numpy
%pip install torch
%pip install datasets
%pip install scikit-learn
%pip install tqdm
%pip install peft
%pip install evaluate
%pip install bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 15.5 MB/s eta 0:00:00


In [21]:
import pandas as pd
import numpy as np
import torch
import bitsandbytes as bnb
from tqdm import tqdm

In [22]:
from datasets import load_dataset

ds = load_dataset("badmatr11x/hate-offensive-speech")

splits = ['train','test','validation']

ds['train']
ds['train'][0]

{'label': 2,
 'tweet': "where's his other half??? i want to indulge myself with a lot of yunho esp him dancing keep you head up today!   â\x9d¤ï¸\x8fð\x9f\x90»ð\x9f\x8d\x93"}

In [4]:
label2id = {"hate-speech": 0, "offensive-speech": 1, "neither": 2}
id2label = {0: "hate-speech", 1: "offensive-speech", 2: "neither"}

# Encode labels
def encode_labels(example):
    label = example['label']
    # Handle both string and integer labels
    if isinstance(label, int):
        label = id2label[label]
    example['label'] = label2id[label]
    return example

encoded_dataset = ds.map(encode_labels)


Map:   0%|          | 0/51070 [00:00<?, ? examples/s]

Map:   0%|          | 0/2838 [00:00<?, ? examples/s]

Map:   0%|          | 0/2837 [00:00<?, ? examples/s]

In [46]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer


model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)

def tokenize_function(examples):
    tokenized_inputs = tokenizer(examples['tweet'], padding="max_length", truncation=True, max_length=512)
    tokenized_inputs['labels'] = examples['label']
    return tokenized_inputs

tokenized_datasets = encoded_dataset.map(tokenize_function, batched=True)
foundation_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

Map:   0%|          | 0/51070 [00:00<?, ? examples/s]

Map:   0%|          | 0/2838 [00:00<?, ? examples/s]

Map:   0%|          | 0/2837 [00:00<?, ? examples/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
from transformers import Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate
import numpy as np

# Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=2,
    weight_decay=0.01,
    learning_rate=5e-5,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    report_to="none"
)

# Load the metric
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# Initialize the Trainer
def train_and_evaluate(model, config_name):
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=tokenized_datasets["train"],
        eval_dataset=tokenized_datasets["test"],
        tokenizer=tokenizer,
        data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
        compute_metrics=compute_metrics,
    )

    trainer.train()
    eval_results = trainer.evaluate()
    print(f"{config_name} evaluation results:", eval_results)
    return eval_results

# Print evaluation results

foundation_eval_results = train_and_evaluate(foundation_model, "Foundation model")



/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,0.227600,0.199454,0.940078
2,0.157200,0.214056,0.946422


Foundation model evaluation results: {'eval_loss': 0.21405577659606934, 'eval_accuracy': 0.9464222770532252, 'eval_runtime': 12.7193, 'eval_samples_per_second': 223.047, 'eval_steps_per_second': 27.91, 'epoch': 2.0}


## Performing Parameter-Efficient Fine-Tuning

TODO: In the cells below, create a PEFT model from your loaded model, run a training loop, and save the PEFT model weights.

In [36]:
from peft import LoraConfig, get_peft_model, TaskType, AutoPeftModelForSequenceClassification

lora_config = LoraConfig(
    task_type=TaskType.SEQ_CLS,
    r=16,
    target_modules=["attention.q_lin", "attention.v_lin"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
)


In [34]:
peft_model = get_peft_model(foundation_model, lora_config)
lora_eval_results = train_and_evaluate(peft_model, "LoRA model")

# Save the LoRA model weights
peft_model.save_pretrained("./peftmodel_weights")



Epoch,Training Loss,Validation Loss,Accuracy
1,0.116300,0.218708,0.945717
2,0.124900,0.215879,0.946070


LoRA model evaluation results: {'eval_loss': 0.21587862074375153, 'eval_accuracy': 0.9460697920338386, 'eval_runtime': 13.0369, 'eval_samples_per_second': 217.613, 'eval_steps_per_second': 27.23, 'epoch': 2.0}


## Performing Inference with a PEFT Model

TODO: In the cells below, load the saved PEFT model weights and evaluate the performance of the trained PEFT model. Be sure to compare the results to the results from prior to fine-tuning.

In [42]:
inference_peft_model = AutoPeftModelForSequenceClassification.from_pretrained("./peftmodel_weights", num_labels=3)
I_peft_model = get_peft_model(inference_peft_model, lora_config)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Loading adapter weights from ./peftmodel_weights led to unexpected keys not found in the model:  ['classifier.modules_to_save.default.base_layer.bias', 'classifier.modules_to_save.default.base_layer.weight', 'classifier.modules_to_save.default.lora_A.default.weight', 'classifier.modules_to_save.default.lora_B.default.weight', 'classifier.modules_to_save.default.modules_to_save.lora_A.default.weight', 'classifier.modules_to_save.default.modules_to_save.lora_B.default.weight', 'classifier.modules_to_save.default.original_module.lora_A.default.weight', 'classifier.modules_to_save.default.original_module.lora_B.default.weight']. 


In [43]:
peft_trainer = Trainer(
    model=I_peft_model,
    args=training_args,
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    data_collator=DataCollatorWithPadding(tokenizer=tokenizer),
    compute_metrics=compute_metrics,
)

# Evaluate the PEFT model
peft_eval_results = peft_trainer.evaluate()


In [44]:
print("Comparison of evaluation results:")
print("Foundation model:", foundation_eval_results)
print("Peft Model:", peft_eval_results)


Comparison of evaluation results:
Foundation model: {'eval_loss': 0.21405577659606934, 'eval_accuracy': 0.9464222770532252, 'eval_runtime': 12.7193, 'eval_samples_per_second': 223.047, 'eval_steps_per_second': 27.91, 'epoch': 2.0}
Peft Model: {'eval_loss': 1.1033282279968262, 'eval_accuracy': 0.276348255199154, 'eval_runtime': 13.1603, 'eval_samples_per_second': 215.573, 'eval_steps_per_second': 26.975}
